In [1]:
from rosette import Rosette
import pyvista as pv
import numpy as np
import itertools
import cv2
import os

In [2]:
a, c, r0, h0, hp, n_arms = 0.5, 2.25, 1.0, 0.25, 0.75, 9
basal_min, basal_max = 0.75, 1.25
prism_min, prism_max = 0.5, 1.5
scaling_range = [basal_min, basal_max, prism_min, prism_max]
solid_angle = 30
ros = Rosette(a, c, r0, h0, hp, n_arms)
ros.unify_mesh()
pl1 = ros.plot()
pl1.enable_parallel_projection()
pl1.remove_all_lights()
pl1.show()

Widget(value="<iframe src='http://localhost:64382/index.html?ui=P_0x1042bb580_0&reconnect=auto' style='width: …

In [3]:
ros_rand = ros.randomize_bullets(scaling=scaling_range, location=solid_angle, inplace=False)
ros_rand.unify_mesh()
pl2 = ros_rand.plot()
pl2.enable_parallel_projection()
pl2.remove_all_lights()
pl2.show()

Widget(value="<iframe src='http://localhost:64382/index.html?ui=P_0x105ba56f0_1&reconnect=auto' style='width: …

In [4]:
# Generate random views of the particle and save as png
save_path = '/Users/josephko/research/ice_renders/test'
n_renders = 100
res = 64
bg_color='black'
obj_color='white'
op=0.9

pl = pv.Plotter(off_screen=True, window_size=[res, res])
pl.background_color = bg_color
pl.enable_parallel_projection()
pl.remove_all_lights()

for i in range(n_renders):
    ros_rot = ros_rand.random_rotate()
    mesh = ros_rot.model
    pl.add_mesh(mesh, show_edges=None, color = obj_color, opacity=op, name='mesh')
    file_name = f'ros_n{n_arms}_a{a}_c{c}_{i}.png'
    file_path = save_path + '/' + file_name
    # pl.show(auto_close=False, interactive=False, jupyter_backend='none')
    # pl.show(screenshot=file_path, interactive=False, jupyter_backend='none')
    pl.screenshot(file_path, return_img=False)
    # pl.deep_clean()
pl.close()

In [ ]:
# Create grid of rendered rosettes
# Source: https://stackoverflow.com/questions/50881227/display-images-in-a-grid
dirname = '/Users/josephko/research/ice_renders' #Name of the directory containing the images
name = dirname +f'/ros_grid_{res}.png' #Name of the exported file
margin = 20 #Margin between pictures in pixels
w = 10 # Width of the matrix (nb of images)
h = 10 # Height of the matrix (nb of images)
n = w*h

filename_list = []

for file in os.listdir(save_path):
    if file.endswith(".png"):
        filename_list.append(file)

filename_list.sort()

imgs = [cv2.imread(save_path+"/"+file) for file in filename_list]

#Define the shape of the image to be replicated (all images should have the same shape)
img_h, img_w, img_c = imgs[0].shape

#Define the margins in x and y directions
m_x = margin
m_y = margin

#Size of the full size image
# mat_x = img_w * w + m_x * (w - 1)
# mat_y = img_h * h + m_y * (h - 1)
mat_x = 720
mat_y = 720

#Create a matrix of zeros of the right size and fill with 255 (so margins end up white)
imgmatrix = np.zeros((mat_y, mat_x, img_c),np.uint8)
imgmatrix.fill(255)

#Prepare an iterable with the right dimensions
positions = itertools.product(range(h), range(w))

for (y_i, x_i), img in zip(positions, imgs):
    x = x_i * (img_w + m_x)
    y = y_i * (img_h + m_y)
    imgmatrix[y:y+img_h, x:x+img_w, :] = img

resized = cv2.resize(imgmatrix, (mat_x//3,mat_y//3), interpolation = cv2.INTER_AREA)
compression_params = [cv2.IMWRITE_JPEG_QUALITY, 90]
cv2.imwrite(name, resized, compression_params)

In [ ]:
# Create grid of CPI images for reference
dirname = '/Users/josephko/research/ICDCdataset/Ros' #Name of the directory containing the images
save_folder = '/Users/josephko/research/ice_renders'
name = save_folder +f'/cpi_grid.png' #Name of the exported file
margin = 20 #Margin between pictures in pixels
w = 10 # Width of the matrix (nb of images)
h = 10 # Height of the matrix (nb of images)
n = w*h

filename_list = []

for file in os.listdir(dirname):
    if file.endswith(".png"):
        filename_list.append(file)

filename_list.sort()

imgs = [cv2.imread(dirname+"/"+file) for file in filename_list]
n_imgs = w*h
imgs = imgs[:n_imgs]

imgs_resized = []
for image in imgs: 
    resized = cv2.resize(image, (256,256), interpolation = cv2.INTER_AREA)
    imgs_resized.append(resized)

#Define the shape of the image to be replicated (all images should have the same shape)
img_h, img_w, img_c = imgs_resized[0].shape

#Define the margins in x and y directions
m_x = margin
m_y = margin

#Size of the full size image
mat_x = img_w * w + m_x * (w - 1)
mat_y = img_h * h + m_y * (h - 1)

#Create a matrix of zeros of the right size and fill with 255 (so margins end up white)
imgmatrix = np.zeros((mat_y, mat_x, img_c),np.uint8)
imgmatrix.fill(255)

#Prepare an iterable with the right dimensions
positions = itertools.product(range(h), range(w))

for (y_i, x_i), img in zip(positions, imgs_resized):
    x = x_i * (img_w + m_x)
    y = y_i * (img_h + m_y)
    imgmatrix[y:y+img_h, x:x+img_w, :] = img

resized = cv2.resize(imgmatrix, (mat_x//3,mat_y//3), interpolation = cv2.INTER_AREA)
compression_params = [cv2.IMWRITE_JPEG_QUALITY, 90]
cv2.imwrite(name, resized, compression_params)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(imgs_resized[5])

In [ ]:
X = np.random.uniform(low=0,high=360,size=(100,3))


In [ ]:
X[0]

In [ ]:
# folder to save images
save_path = '/Users/josephko/research/ice_renders/sample_rosettes'

for a in a_list:
    for c in c_list:
        for n_arms in n_arms_list:
            ros = Rosette(a, c, r0, h0, hp, n_arms)
            ros_rotated = ros.random_rotate()
            pl = ros_rotated.plot()
            file_name = f'ros_n{n_arms}_a{a}_c{c}.png'
            file_path = save_path + '/' + file_name
            pl.show(screenshot=file_path, interactive=False, jupyter_backend='none')